# Example of using Spark SQL and SQL Queries

In [33]:
# Prerequisites
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [34]:
# Get SparkSession
spark = SparkSession.builder.master("local").getOrCreate()
print("Spark Version: ", spark.version)

Spark Version:  3.5.0


## Read in flight delay dataset

In [37]:
file_path = "data/departure_delays.csv"
df = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(file_path))

df.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [41]:
df.count()

1391578

In [40]:
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [ ]:
# Create temporary view to issue SQL queries using Spark SQL
df.createOrReplaceTempView("us_delay_flights_tbl")

In [51]:
# Show all
spark.sql("SELECT * FROM us_delay_flights_tbl").show(n=10)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 10 rows



## Some Basic Queries

Find all flights with distance greater than 1,000 miles

In [42]:
spark.sql("""SELECT distance, origin, destination 
          FROM us_delay_flights_tbl WHERE distance > 1000 
          ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [48]:
# The same as DataFrame API Query
(df.select("distance", "origin", "destination")
    .where(col("distance") > 1000)
    .orderBy(desc("distance"))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



Find all flights between SFO and ORD with at least 2 hour delay

In [44]:
spark.sql("""SELECT date, delay, origin, destination 
          FROM us_delay_flights_tbl 
          WHERE delay > 120 AND origin == 'SFO' AND destination == 'ORD'
          ORDER BY delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



Label all flights, regardless of origin and destination, with an indication of the delay severity: Very Long Delays (> 6 hours), Long Delays (2–6 hours), etc. Add a new column called Flight_Delays:

In [46]:
spark.sql("""SELECT delay, origin, destination, 
          CASE
            WHEN delay > 360 THEN 'Very Long Delays'
            WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
            WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
            WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delays'
            WHEN delay == 0 THEN 'No Delays'
            ELSE 'Early'
          END AS Flight_Delays
          FROM us_delay_flights_tbl 
          ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



## Creating SQL Databases and Tables

In [81]:
# Create databased with your own name
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

### Create a Managed Table with SQL commands

In [82]:
spark.sql("CREATE TABLE managed_learn_spark_db_tbl (date STRING, delay INT)")


AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`learn_spark_db`.`managed_learn_spark_db_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


### Creating a Managed Table

In [59]:
# Creating Managed table with DataFrame API
file_path = "data/departure_delays.csv"
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
df_flights = spark.read.csv(file_path, schema=schema)
df_flights.write.saveAsTable("managed_us_delay_flights_tbl")


AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`learn_spark_db`.`managed_us_delay_flights_tbl` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

### Creating an Unmanaged Table

In [ ]:
# Creating an Unmanaged table with SQL API
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, 
  distance INT, origin STRING, destination STRING) 
  USING csv OPTIONS (PATH '/data/departure_delays.csv')""")

In [ ]:
# Creating an Unmanaged table with DataFrame API
(df_flights
  .write
  .option("path", "/tmp/data/us_flights_delay")
  .saveAsTable("us_delay_flights_tbl"))

## Creating Views
Spark can create views on top of existing tables. Views can be global (visible across all SparkSessions on a given cluster) or session-scoped (visible only to a single SparkSession).<br />

The difference between a view and a table is that views don’t actually hold the data; tables persist after your Spark application terminates, but views disappear.


In [72]:
df_sfo = spark.sql("""SELECT date, delay, origin, destination FROM 
  us_delay_flights_tbl WHERE origin = 'SFO'""")
df_jfk = spark.sql("""SELECT date, delay, origin, destination FROM 
  us_delay_flights_tbl WHERE origin = 'JFK'""")

# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

Once you’ve created these views, you can issue queries against them just as you would against a table.


In [74]:
spark.read.table("us_origin_airport_JFK_tmp_view").show(5, False)

+-------+-----+------+-----------+
|date   |delay|origin|destination|
+-------+-----+------+-----------+
|1010900|14   |JFK   |LAX        |
|1011200|-3   |JFK   |LAX        |
|1011900|2    |JFK   |LAX        |
|1011700|11   |JFK   |LAS        |
|1010800|-1   |JFK   |SFO        |
+-------+-----+------+-----------+
only showing top 5 rows



In [75]:
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



In [76]:
# Drop Views
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

True

## Viewing Metadata
Spark manages the metadata associated with each managed or unmanaged table. This is captured in the Catalog.

In [66]:
# List databases
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/jovyan/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/home/jovyan/spark-warehouse/learn_spark_db.db')]

In [67]:
# List tables
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [70]:
# List columns
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]